In [ ]:
!pip install sentence-transformers==2.7.0
!pip install rank-bm25
!pip install sastrawi
!pip install swifter

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from sentence_transformers import SentenceTransformer,util
import pandas as pd
import pickle

import string
import re

from statistics import mean
from rank_bm25 import BM25Okapi

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Dataset

In [ ]:
dfAns = pd.read_csv('/content/drive/MyDrive/Thesis/Dataset KUP Preprocessed.csv', sep=';', dtype={'Angka':'str','Paragraf':'str'})

dfSearch = dfAns[['Bab', 'State', 'Pasal', 'Ayat', 'Angka', 'Huruf', 'Paragraf', 'Isi']].copy()
dfSearch['Isi_lower'] = dfSearch['Isi'].apply(lambda x: x.lower())
dfSearch['Pasal'] = dfSearch['Pasal'].apply(lambda x: x.lower())
dfSearch['Ayat'] = dfSearch['Ayat'].apply(lambda x: x.lower())

dfSearch[~(dfSearch['Isi_lower'] == 'dihapus.')]
dfSearch = dfSearch.reset_index(drop=True)

# Load Model

In [ ]:
model_name = 'fine_tuned_model_kup_all_a1'
model_multimpnet = SentenceTransformer("/content/drive/MyDrive/Thesis_Finetuned/"+model_name)

with open("/content/drive/MyDrive/Thesis_Finetuned/multilingual_mpnet_embeddings_"+model_name+".pkl", "rb") as fIn:
  cache_data = pickle.load(fIn)
  multilingual_mpnet_embeddings = cache_data['multilingual_mpnet_embeddings']

model_multimpnet_raw = SentenceTransformer("/content/drive/MyDrive/Thesis_Finetuned/paraphrase-multilingual-mpnet-base-v2")

with open("/content/drive/MyDrive/Thesis/multilingual_mpnet_embeddings_raw.pkl", "rb") as fIn:
  cache_data = pickle.load(fIn)
  multilingual_mpnet_embeddings_raw = cache_data['multilingual_mpnet_embeddings_raw']

In [ ]:
corpus = dfAns['Isi_lower']
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Load Kamus Normalisasi

In [ ]:
with open("/content/drive/MyDrive/Thesis/kamus_normalisasi.pkl", "rb") as kamus:
  cache_data = pickle.load(kamus)
  tax_term_dict = cache_data['kamus']

In [ ]:
def search_query(query, model, embeddings, column_name):
  query_embedding = model.encode(query)
  similarity = util.cos_sim(query_embedding, embeddings)
  index_score = [i.item() for i in similarity[0]]
  dfAns[column_name] = index_score

def reciprocal_rank_fusion(result):
  k = 60
  return sum([1.0 / (k + i) for i in result])

def normalization_query(query):
  for tax_term in tax_term_dict:
    query = re.sub(r"\W"+tax_term+"\W", " "+tax_term_dict[tax_term] + " ", query)
    query = re.sub(r"\A"+tax_term+" ", tax_term_dict[tax_term] + " ", query)
    query = re.sub(r" "+tax_term+"\Z", " " + tax_term_dict[tax_term], query)
    query = re.sub(r"\A"+tax_term+"\Z", tax_term_dict[tax_term], query)
    query = re.sub(r"\W"+tax_term.upper()+"\W", " "+tax_term_dict[tax_term] + " ", query)
    query = re.sub(r"\A"+tax_term.upper()+" ", tax_term_dict[tax_term] + " ", query)
    query = re.sub(r" "+tax_term.upper()+"\Z", " " + tax_term_dict[tax_term], query)
    query = re.sub(r"\A"+tax_term.upper()+"\Z", tax_term_dict[tax_term], query)
  return query

# Memulai Pencarian

In [ ]:
query_asli = str(input('Masukkan kueri pencarian: '))
query = normalization_query(query_asli)
query = query.lower()
print('Kueri dinormalisasi:', query)

dfAns.drop(dfAns.columns[14:], axis=1, inplace=True)
search_query(query = query, model = model_multimpnet, embeddings = multilingual_mpnet_embeddings, column_name = 'multi_mpnet')
search_query(query = query, model = model_multimpnet_raw, embeddings = multilingual_mpnet_embeddings_raw, column_name = 'multi_mpnet_raw')

tokenized_query = query.split(" ")
doc_scores = bm25.get_scores(tokenized_query)
dfAns['BM25'] = doc_scores

dfAns2 = dfAns.copy()
dfAns2['index_corpus'] = dfAns2.index

dfAns2 = dfAns2.sort_values(by='multi_mpnet_raw', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_multi_mpnet_raw'] = dfAns2.index + 1

dfAns2 = dfAns2.sort_values(by='multi_mpnet', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_multi_mpnet'] = dfAns2.index + 1

dfAns2 = dfAns2.sort_values(by='BM25', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_BM25'] = dfAns2.index + 1

dfAns2['RRF_raw'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_multi_mpnet_raw'], x['index_BM25']]), axis=1)
dfAns2 = dfAns2.sort_values(by='RRF_raw', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_RRF_raw'] = dfAns2.index + 1

dfAns2['RRF'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_multi_mpnet'], x['index_BM25']]), axis=1)
dfAns2 = dfAns2.sort_values(by='RRF', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_RRF'] = dfAns2.index + 1

dfOut = dfAns2.sort_values(by='index_RRF')[['Pasal','Ayat', 'Angka','Isi', 'index_RRF']].head(10)

# Contoh Kueri
# Pemberlakukan NIK sebagai nomor identitas wajib pajak
# Setiap Wajib Pajak wajib membayar pajak

Masukkan kueri pencarian: Pemberlakukan NIK sebagai nomor identitas wajib pajak
Kueri dinormalisasi: pemberlakukan nomor induk kependudukan sebagai nomor pokok wajib pajak


# Cetak Hasil Pencarian

In [ ]:
pd.set_option('display.max_colwidth', 1000)
dfOut.set_index('index_RRF')

,Pasal,Ayat,Angka,Isi
index_RRF,,,,
1,2,10,0,"Dalam rangka penggunaan nomor induk kependudukan sebagai Nomor Pokok Wajib Pajak sebagaimana dimaksud pada ayat (1a), menteri yang menyelenggarakan urusan pemerintahan dalam negeri memberikan data kependudukan dan data balikan dari pengguna kepada Menteri Keuangan untuk diintegrasikan dengan basis data perpajakan."
2,2,1a,0,Nomor Pokok Wajib Pajak sebagaimana dimaksud pada ayat (1) bagi Wajib Pajak orang pribadi yang merupakan penduduk Indonesia menggunakan nomor induk kependudukan.
3,1,0,6,Nomor Pokok Wajib Pajak adalah nomor yang diberikan kepada Wajib Pajak sebagai sarana dalam administrasi perpajakan yang dipergunakan sebagai tanda pengenal diri atau identitas Wajib Pajak dalam melaksanakan hak dan kewajiban perpajakannya.
4,37A,2,0,"Wajib Pajak orang pribadi yang secara sukarela mendaftarkan diri untuk memperoleh Nomor Pokok Wajib Pajak paling lama 1 (satu) tahun setelah berlakunya Undang-Undang ini diberikan penghapusan sanksi administrasi atas pajak yang tidak atau kurang dibayar untuk Tahun Pajak sebelum diperoleh Nomor Pokok Wajib Pajak dan tidak dilakukan pemeriksaan pajak, kecuali terdapat data atau keterangan yang menyatakan bahwa Surat Pemberitahuan yang disampaikan Wajib Pajak tidak benar atau menyatakan lebih bayar."
5,44E,2,0,"Ketentuan lebih lanjut mengenai: a. jangka waktu pendaftaran dan pelaporan serta tata cara pendaftaran dan pengukuhan sebagaimana dimaksud dalam Pasal 2 ayat (1), ayat (2), ayat (3), dan ayat (4) termasuk penggunaan nomor induk kependudukan sebagai Nomor Pokok Wajib Pajak, penghapusan Nomor Pokok Wajib Pajak dan/atau pencabutan Pengukuhan Pengusaha Kena Pajak; b. pemberian dan permintaan bantuan penagihan pajak sebagaimana dimaksud dalam Pasal 20A ayat (2); c. penampungan dan pengiriman hasil penagihan pajak atas klaim pajak sebagaimana dimaksud dalam Pasal 20A ayat (9); d. pelaksanaan prosedur persetujuan bersama sebagaimana dimaksud dalam Pasal 27C ayat (1); e. pelaksanaan hak dan pemenuhan kewajiban perpajakan oleh seorang kuasa sebagaimana dimaksud dalam Pasal 32 ayat (3) serta kompetensi tertentu yang harus dimiliki seorang kuasa sebagaimana dimaksud dalam Pasal 32 ayat (3a); f. penunjukan, pemotongan, pemungutan, penyetoran, dan/atau pelaporan pajak yang telah dipotong atau d..."
6,2,4a,0,"Kewajiban perpajakan bagi Wajib Pajak yang diterbitkan Nomor Pokok Wajib Pajak dan/atau yang dikukuhkan sebagai Pengusaha Kena Pajak secara jabatan sebagaimana dimaksud pada ayat (4) dimulai sejak saat Wajib Pajak memenuhi persyaratan subjektif dan objektif sesuai dengan ketentuan peraturan perundang-undangan perpajakan, paling lama 5 (lima) tahun sebelum diterbitkannya Nomor Pokok Wajib Pajak dan/atau dikukuhkannya sebagai Pengusaha Kena Pajak."
7,2,6,0,Penghapusan Nomor Pokok Wajib Pajak dilakukan oleh Direktur Jenderal Pajak apabila: a. diajukan permohonan penghapusan Nomor Pokok Wajib Pajak oleh Wajib Pajak dan/atau ahli\nwarisnya apabila Wajib Pajak sudah tidak memenuhi persyaratan subjektif dan/atau objektif sesuai dengan ketentuan peraturan perundang-undangan perpajakan; b. Wajib Pajak badan dilikuidasi karena penghentian atau penggabungan usaha; c. Wajib Pajak bentuk usaha tetap menghentikan kegiatan usahanya di Indonesia; atau d. dianggap perlu oleh Direktur Jenderal Pajak untuk menghapuskan Nomor Pokok Wajib Pajak dari Wajib Pajak yang sudah tidak memenuhi persyaratan subjektif dan/atau objektif sesuai dengan ketentuan peraturan perundang-undangan perpajakan.
8,2,4,0,Direktur Jenderal Pajak menerbitkan Nomor Pokok Wajib Pajak dan/atau mengukuhkan Pengusaha Kena Pajak secara jabatan apabila Wajib Pajak atau Pengusaha Kena Pajak tidak melaksanakan kewajibannya sebagaimana dimaksud pada ayat (1) dan/atau ayat (2).
9,2,1,0,Setiap Wajib Pajak yang telah memenuhi persyaratan subjektif dan objektif sesuai dengan ketentuan peraturan perundang-undangan perpajakan wajib mendaftarkan diri pada kantor Direktorat Jenderal Pajak yang wilayah kerjanya melip